In [ ]:
# Import required libraries
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

# Load CIFAR-10 dataset
cifar10 = fetch_openml(name='CIFAR_10')

# Extract features and labels
X = cifar10.data.astype('float32')   # X contains all the image data 
y = cifar10.target.astype('int')     # y contains the class labels for each image 

# Split the dataset into train and test sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features using StandardScaler method
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert labels to one-hot encoding format using np.eye method
num_classes = 10
y_train_onehot = np.eye(num_classes)[y_train]



In [ ]:
# Define a class RBFN 
class RBFN:
    def __init__(self, num_hidden, sigma=2.0):
        self.num_hidden = num_hidden    # Number of RBF centers
        self.sigma = sigma              # Width of RBF kernel
        self.centers = None             # Center points for RBF kernels
        self.weights = None             # Weights for linear combination of RBF kernels

    # Radial basis function that computes the distance between input data and centroids
    def _rbf_kernel(self, X, centers):
        return np.exp(-np.linalg.norm(X - centers, axis=1) ** 2 / (2 * self.sigma ** 2))

    # One-hot encoding of labels
    def _onehot_encoding(self, y):
        return np.eye(num_classes)[y]

    # Softmax function to compute output probabilities
    def _softmax(self, X):
        exp_scores = np.exp(X)
        return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Fit function that trains the RBF network
    def fit(self, X, y, num_epochs=20, learning_rate=0.1):
        # Initialize RBF centers randomly from input data
        self.centers = X[np.random.choice(X.shape[0], self.num_hidden), :]
        # Initialize RBF weights
        self.weights = np.random.randn(self.num_hidden, num_classes)

        # Train the model using gradient descent
        for epoch in range(num_epochs):
            # Forward pass
            # Compute hidden layer using RBF kernel
            hidden_layer = self._rbf_kernel(X, self.centers)
            # Compute output layer using dot product of hidden layer with weights
            output_layer = np.dot(hidden_layer, self.weights)
            # Compute predicted output probabilities using softmax function
            predicted_probs = self._softmax(output_layer)

            # Backward pass
            # Compute gradient of output layer
            grad_output = predicted_probs
            grad_output[range(X.shape[0]), y] -= 1
            grad_output /= X.shape[0]

            # Compute gradient of weights and hidden layer
            grad_weights = np.dot(hidden_layer.T, grad_output)
            grad_hidden = np.dot(grad_output, self.weights.T)

            # Update weights and centers using gradients and learning rate
            self.weights -= learning_rate * grad_weights
            self.centers -= learning_rate * (grad_hidden * hidden_layer * (X - self.centers) / (self.sigma ** 2))

    # Predict function that uses the trained RBF network to predict classes 
    def predict(self, X):
        hidden_layer = self._rbf_kernel(X, self.centers)
        output_layer = np.dot(hidden_layer, self.weights)
        return np.argmax(output_layer, axis=1)

# Create an instance of RBFN model with 20 RBF centers
rbfnet = RBFN(num_hidden=20)


In [ ]:
rbfnet.fit(X_train, y_train, num_epochs=20, learning_rate=0.1)


In [ ]:
# Predict labels for the training and test sets
y_train_pred = rbfnet.predict(X_train)
y_test_pred = rbfnet.predict(X_test)

# Calculate accuracy, F1 score, and confusion matrix
train_accuracy = accuracy_score(y_train, y_train_pred)
train_f1_score = f1_score(y_train, y_train_pred, average='macro')
train_confusion_matrix = confusion_matrix(y_train, y_train_pred)

test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1_score = f1_score(y_test, y_test_pred, average='macro')
test_confusion_matrix = confusion_matrix(y_test, y_test_pred)

# Print the results of evaluation metrics
print('Training Accuracy: %.2f %%' % (train_accuracy * 100))
print('Training F1 Score: %.4f' % train_f1_score)
print('Training Confusion Matrix:\n', train_confusion_matrix)

print('Test Accuracy: %.2f %%' % (test_accuracy * 100))
print('Test F1 Score: %.4f' % test_f1_score)
print('Test Confusion Matrix:\n', test_confusion_matrix)
